In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

In [135]:
def make_timedelta(df):
    df['seconds'] = df['time']
    df['time'] = pd.to_timedelta(df['time'], unit='s')

def get_highest_accuracy(df):
    highest_accuracy = pd.DataFrame(columns=df.columns)
   
    i = 0
    for er in np.unique(df["error-rate"]):
        for j in range(len(df[df["matches"] == np.max(df[df["error-rate"] == er]["matches"])])):
            highest_accuracy.loc[i + j] = list(df[df["matches"] == np.max(df[df["error-rate"] == er]["matches"])].iloc[j])
            i = i + 1
    
    return highest_accuracy

def get_lowest_memory(df):
    lowest_memory = pd.DataFrame(columns=df.columns)
   
    i = 0
    for er in np.unique(df["error-rate"]):
        for j in range(len(df[df["mem"] == np.min(df[df["error-rate"] == er]["mem"])])):
            lowest_memory.loc[i + j] = list(df[df["mem"] == np.min(df[df["error-rate"] == er]["mem"])].iloc[j])
            i = i + 1
    
    return lowest_memory
    
def get_fewest_matches(df):
    fewest_matches = pd.DataFrame(columns=df.columns)
   
    i = 0
    for er in np.unique(df["error-rate"]):
        for j in range(len(df[df["matches"] == np.min(df[df["error-rate"] == er]["matches"])])):
            fewest_matches.loc[i + j] = list(df[df["matches"] == np.min(df[df["error-rate"] == er]["matches"])].iloc[j])
        i = i + 1
    
    return fewest_matches


def get_fastest(df):
    fastest = pd.DataFrame(columns=df.columns)
   
    i = 0
    for er in np.unique(df["error-rate"]):
        for j in range(len(df[df["seconds"] == np.max(df[df["error-rate"] == er]["seconds"])])):
            fastest.loc[i + j] = list(df[df["seconds"] == np.max(df[df["error-rate"] == er]["seconds"])].iloc[j])
            i = i + 1
    
    return fastest

# Mouse vs fly reference genomes

min_len = 150

## Stellar

In [165]:
stellar_time = pd.read_csv("stellar_mouse.time", sep = "\t")
make_timedelta(stellar_time)

In [166]:
stellar_matches = pd.read_csv("stellar_match_count.tsv", sep = "\t")
#stellar_matches.head()

In [167]:
stellar_mouse = pd.merge(stellar_time, stellar_matches, how="left", on=["error-rate", "repeat-period", "repeat-length", "min-len"])
#stellar_mouse.head()

In [171]:
stellar_mouse[(stellar_mouse["error-rate"] > 0.4)]

,time,mem,error-code,command,min-len,error-rate,repeat-period,repeat-length,seconds,matches


In [157]:
failed_runs = stellar_mouse[(stellar_mouse["error-code"] != 0) & (stellar_mouse["matches"] > 0)].index
stellar_mouse.drop(failed_runs, inplace = True)

In [158]:
wrong_error_code = stellar_mouse[(stellar_mouse["error-code"] == 0) & (stellar_mouse["matches"] == 0)].index
stellar_mouse.drop(wrong_error_code, inplace = True)

In [159]:
stellar_mouse = stellar_mouse[stellar_mouse["error-code"] == 0]
error_rates = np.unique(stellar_mouse["error-rate"])
#stellar_mouse.head()

In [160]:
for er in error_rates:
    er_rate_runs = stellar_mouse[stellar_mouse["error-rate"] == er]
    print(er)
    print("Corr repeat-period\t" + str(er_rate_runs["repeat-period"].corr(er_rate_runs["matches"])))
    print("Corr repeat-length\t" + str(er_rate_runs["repeat-length"].corr(er_rate_runs["matches"])))
    print("Corr time\t" + str(er_rate_runs["time"].corr(er_rate_runs["matches"])))

0.0267
Corr repeat-period	-0.5749652444761867
Corr repeat-length	0.2869991179202832
Corr time	0.6783328354499112
0.033
Corr repeat-period	-0.5188064722046153
Corr repeat-length	0.26764405714678297
Corr time	0.7731811961995414
0.04
Corr repeat-period	-0.4415139176838639
Corr repeat-length	0.23558846650590795
Corr time	0.6670797876432318


In [161]:
stellar_default = stellar_mouse[(stellar_mouse["repeat-period"] == 1) & (stellar_mouse["repeat-length"] == 1000)]
stellar_default

,time,mem,error-code,command,min-len,error-rate,repeat-period,repeat-length,seconds,matches
0,0 days 02:08:42,10005920,0,stellar-search,150,0.0267,1,1000,7722.00,103208
6,0 days 06:07:46.030000,10307376,0,stellar-search,150,0.0330,1,1000,22066.03,119991
43,0 days 15:30:48.120000,10419560,0,stellar-search,150,0.0400,1,1000,55848.12,160137


In [29]:
stellar_true_matches = stellar_default[["error-rate", "matches"]] 
stellar_true_matches.columns = ["error-rate", "stellar-matches"]
stellar_true_matches

,error-rate,stellar-matches
0,0.0267,103208
6,0.0330,119991
43,0.0400,160137


In [164]:
acc_dream = get_highest_accuracy(dream_mouse)
acc_dream.drop(["cmax", "cart-max-cap", "max-carts"], axis = 1, inplace = True)
acc_dream

,time,mem,error-code,command,bins,max-er,min-len,threads,error-rate,bin-entropy-cutoff,repeats,repeat-period,repeat-length,matches,repeat-mask,kmer-size,seconds,stellar-matches
0,0 days 00:00:31.180000,11811916,0,valik-search,1024,0.0067,150,32,0.0067,0.75,45,1000.0,NaN,22927,entropy,NaN,31.18,0
1,0 days 00:00:45.390000,13877048,0,valik-search,1024,0.0130,150,32,0.0130,0.95,45,4.0,1000.0,38553,entropy,NaN,45.39,0
2,0 days 00:01:04.260000,14216480,0,valik-search,1024,0.0200,150,32,0.0200,0.95,54,4.0,1000.0,64408,entropy,NaN,64.26,0
3,0 days 00:01:37.550000,13591332,0,valik-search,1024,0.0267,150,32,0.0267,0.95,76,1.0,1000.0,83772,entropy,NaN,97.55,103208
4,0 days 00:01:49.340000,13461092,0,valik-search,1024,0.0330,150,32,0.0330,0.95,76,1.0,1000.0,100694,entropy,NaN,109.34,119991
5,0 days 00:03:13.080000,15874196,0,valik-search,1024,0.0400,150,32,0.0400,0.95,139,1.0,1000.0,138123,entropy,NaN,193.08,160137
6,0 days 01:21:27.780000,318939592,0,valik-search,1024,0.0467,150,32,0.0467,0.75,12151,1000.0,NaN,144958,entropy,NaN,4887.78,0
7,0 days 00:28:21.250000,185189236,0,valik-search,4096,0.0530,150,32,0.0530,0.10,5791,1.0,1000.0,48505,entropy,NaN,1701.25,0
9,0 days 00:28:20.290000,185167760,0,valik-search,4096,0.0530,150,32,0.0530,0.10,5791,1.0,1000.0,48505,entropy,NaN,1700.29,0


In [31]:
get_highest_accuracy(stellar_mouse)

,time,mem,error-code,command,min-len,error-rate,repeat-period,repeat-length,seconds,matches
0,0 days 02:08:42,10005920,0,stellar-search,150,0.0267,1,1000,7722.00,103208
2,0 days 02:07:01.320000,11734356,0,stellar-search,150,0.0267,2,1000,7621.32,103208
4,0 days 02:04:38.160000,10010436,0,stellar-search,150,0.0267,1,25,7478.16,103208
6,0 days 02:06:39.850000,11729160,0,stellar-search,150,0.0267,2,100,7599.85,103208
8,0 days 01:35:05.840000,11728128,0,stellar-search,150,0.0267,2,50,5705.84,103208
10,0 days 00:25:06.880000,11690012,0,stellar-search,150,0.0267,2,25,1506.88,103208
12,0 days 02:05:56.360000,10011380,0,stellar-search,150,0.0267,1,100,7556.36,103208
14,0 days 02:05:34.080000,10009160,0,stellar-search,150,0.0267,1,50,7534.08,103208
16,0 days 02:09:40.190000,11737852,0,stellar-search,150,0.0267,4,1000,7780.19,103208
9,0 days 06:07:46.030000,10307376,0,stellar-search,150,0.0330,1,1000,22066.03,119991


## Blast

In [32]:
print(error_rates)

[0.0267 0.033  0.04  ]


In [33]:
min_len = 150
kmer_list = []
error_list = [0.0067, 0.013, 0.02, 0.0267, 0.033, 0.04, 0.0467, 0.053]
for er in error_list:
    errors = int(round(er * min_len))
    for k in range(51, 6, -1):
        if ((min_len - k + 1 - errors * k) > 2):
            kmer_list.append(k) 
            break
            
d = {'k': kmer_list, 'error-rate': error_list}            
error_df = pd.DataFrame(data=d)
print(error_df)

    k  error-rate
0  51      0.0067
1  49      0.0130
2  37      0.0200
3  29      0.0267
4  24      0.0330
5  21      0.0400
6  18      0.0467
7  16      0.0530


In [34]:
blast_time = pd.read_csv("blast_mouse.time", sep = "\t")
blast_time = blast_time[blast_time["command"] == "blast-search"]
blast_time.threads = blast_time.threads.astype(int)
blast_time.k = blast_time.k.astype(int)
blast_time.drop("matches", axis = 1, inplace = True)
make_timedelta(blast_time)
#blast_time.head()

In [35]:
blast_matches = pd.read_csv("blast_mouse_match_count.tsv", sep = "\t")
#blast_matches.head()

In [36]:
blast_mouse = pd.merge(blast_time, blast_matches, how="left", on=["evalue", "k"])
blast_mouse = pd.merge(blast_mouse, error_df, how="left", on = ["k"])
blast_mouse = pd.merge(blast_mouse, stellar_true_matches, how="left", on = ["error-rate"])
blast_mouse['stellar-matches'] = blast_mouse['stellar-matches'].fillna(0).astype(int)
blast_mouse.head()

,time,mem,error-code,command,threads,evalue,k,seconds,matches,error-rate,stellar-matches
0,0 days 00:01:05.580000,1032700,0,blast-search,8,10.00,49,65.58,809,0.0130,0
1,0 days 00:02:56.420000,1761172,0,blast-search,8,0.10,21,176.42,17607,0.0400,160137
2,0 days 00:01:20.390000,1126004,0,blast-search,8,0.01,37,80.39,1779,0.0200,0
3,0 days 00:04:49.190000,2274744,0,blast-search,8,10.00,18,289.19,185203,0.0467,0
4,0 days 00:10:43.640000,2055048,0,blast-search,8,0.01,16,643.64,27491,0.0530,0


In [37]:
len(blast_mouse[(blast_mouse["error-code"] != 0) & (blast_mouse["matches"] > 0)]) == 0

True

In [38]:
len(blast_mouse[(blast_mouse["error-code"] == 0) & (blast_mouse["matches"] == 0)]) == 0

True

## DREAM-Stellar

In [81]:
dream_entropy_max_ent = pd.read_csv("error_rates/entropy/search_valik_ent0.95.time", sep = "\t")
dream_entropy_max_ent["repeat-mask"] = "entropy"

In [82]:
dream_entropy_max_er_time = pd.read_csv("error_rates/entropy/search_valik_max_er.time", sep = "\t")
dream_entropy_max_er_time["repeat-mask"] = "entropy"
dream_entropy_max_er_time["repeat-period"] = 1
dream_entropy_max_er_time["repeat-period"] = 1000
#dream_entropy_max_er_time.head()

In [83]:
dream_entropy_time = pd.read_csv("error_rates/entropy/search_valik.time", sep = "\t")
dream_entropy_time.drop("matches", axis = 1, inplace = True)
dream_entropy_time["repeat-period"] = 1
dream_entropy_time["repeat-length"] = 1000
#dream_entropy_time.head()

In [84]:
dream_same = dream_entropy_time.drop(["time", "mem", "repeats"], axis = 1)
print(len(dream_same))
dream_same.drop_duplicates()
print(len(dream_same))

86
86


In [85]:
dream_entropy_matches = pd.read_csv("error_rates/entropy/valik_mouse_match_count.tsv", sep = "\t")
dream_entropy_matches = dream_entropy_matches[dream_entropy_matches["repeat-period"] == 1]
dream_entropy_matches = dream_entropy_matches[dream_entropy_matches["repeat-length"] == 1000]
#dream_entropy_matches.head()

In [86]:
dream_entropy_mouse = pd.merge(dream_entropy_time, dream_entropy_matches, how="left", on=["repeat-period", "repeat-length", "bins", "cmax", "error-rate", "bin-entropy-cutoff", "max-carts"])
dream_entropy_mouse["repeat-mask"] = "entropy"
#dream_entropy_mouse.head()

In [87]:
dream_pattern_count_mouse = pd.read_csv("error_rates/pattern_count/search_valik.time", sep = "\t")
dream_pattern_count_mouse["repeat-mask"] = "pattern-count"
#dream_pattern_count_mouse.head()

In [88]:
dream_mouse = pd.concat([dream_entropy_mouse, dream_pattern_count_mouse, dream_entropy_max_er_time, dream_entropy_max_ent])
dream_mouse.drop(["fpr", "minimiser", "repeat-flag", "cmin"], axis = 1, inplace = True)
dream_mouse['repeats'] = dream_mouse['repeats'].fillna(0).astype(int)
dream_mouse['matches'] = dream_mouse['matches'].fillna(0).astype(int)
make_timedelta(dream_mouse)
dream_mouse = pd.merge(dream_mouse, stellar_true_matches, how="left", on = ["error-rate"])
dream_mouse["stellar-matches"] = dream_mouse['stellar-matches'].fillna(0).astype(int)
#dream_mouse.head()

In [104]:
np.mean(dream_mouse[(dream_mouse["error-rate"] == 0.02) & (dream_mouse["bins"] == 2048) & (dream_mouse["threads"] == 32)]["matches"])

26114.75

In [114]:
er_df = dream_mouse[(dream_mouse["error-rate"] == 0.0467) & (dream_mouse["error-code"] ==0) & (dream_mouse["repeat-mask"] == "entropy")].copy()
er_df.drop(["command", "min-len", "cmax", "cart-max-cap"], axis = 1, inplace = True)
er_df

,time,mem,error-code,bins,max-er,threads,error-rate,bin-entropy-cutoff,max-carts,repeats,repeat-period,repeat-length,matches,repeat-mask,kmer-size,seconds,stellar-matches
55,0 days 00:26:34.250000,241078244,0,2048,0.0467,32,0.0467,0.10,2048,9116,1.0,1000.0,43419,entropy,NaN,1594.25,0
59,0 days 00:31:30.210000,272823228,0,2048,0.0467,32,0.0467,0.25,2048,9116,1.0,1000.0,67936,entropy,NaN,1890.21,0
65,0 days 00:14:22.850000,235509104,0,1024,0.0467,32,0.0467,0.10,2048,12151,1.0,1000.0,31840,entropy,NaN,862.85,0
76,0 days 00:14:34.090000,241994500,0,2048,0.0467,32,0.0467,0.10,2048,9116,1.0,1000.0,43419,entropy,NaN,874.09,0
85,0 days 01:33:47.790000,318578188,0,1024,0.0530,32,0.0467,0.75,1024,0,1.0,1000.0,0,entropy,NaN,5627.79,0
115,0 days 01:21:27.780000,318939592,0,1024,0.0467,32,0.0467,0.75,2048,12151,1000.0,NaN,144958,entropy,NaN,4887.78,0


## Results

In [48]:
blast_mouse[blast_mouse["error-rate"] == 0.0067]

,time,mem,error-code,command,threads,evalue,k,seconds,matches,error-rate,stellar-matches
6,0 days 00:00:59.420000,1028648,0,blast-search,8,0.10,51,59.42,690,0.0067,0
13,0 days 00:00:59.070000,1030608,0,blast-search,8,1.00,51,59.07,690,0.0067,0
29,0 days 00:00:59.500000,1027924,0,blast-search,8,10.00,51,59.50,690,0.0067,0
31,0 days 00:00:59.970000,1015984,0,blast-search,8,0.01,51,59.97,690,0.0067,0
38,0 days 00:00:49.740000,1073132,0,blast-search,16,0.10,51,49.74,690,0.0067,0
60,0 days 00:00:56.590000,1092388,0,blast-search,16,1.00,51,56.59,690,0.0067,0
61,0 days 00:00:55.340000,1081980,0,blast-search,16,10.00,51,55.34,690,0.0067,0
63,0 days 00:00:52.030000,1093224,0,blast-search,16,0.01,51,52.03,690,0.0067,0


In [49]:
blast_acc = get_highest_accuracy(blast_mouse)
blast_acc

,time,mem,error-code,command,threads,evalue,k,seconds,matches,error-rate,stellar-matches
0,0 days 00:00:59.420000,1028648,0,blast-search,8,0.10,51,59.42,690,0.0067,0
2,0 days 00:00:59.070000,1030608,0,blast-search,8,1.00,51,59.07,690,0.0067,0
4,0 days 00:00:59.500000,1027924,0,blast-search,8,10.00,51,59.50,690,0.0067,0
6,0 days 00:00:59.970000,1015984,0,blast-search,8,0.01,51,59.97,690,0.0067,0
8,0 days 00:01:05.580000,1032700,0,blast-search,8,10.00,49,65.58,809,0.0130,0
10,0 days 00:01:07.530000,1031132,0,blast-search,8,0.10,49,67.53,809,0.0130,0
12,0 days 00:01:03.990000,1027756,0,blast-search,8,0.01,49,63.99,809,0.0130,0
14,0 days 00:01:06.470000,1029908,0,blast-search,8,1.00,49,66.47,809,0.0130,0
16,0 days 00:01:20.390000,1126004,0,blast-search,8,0.01,37,80.39,1779,0.0200,0
18,0 days 00:01:20.070000,1101496,0,blast-search,8,10.00,37,80.07,1779,0.0200,0


In [99]:
dream_acc = get_highest_accuracy(dream_mouse)
dream_acc.drop(["cart-max-cap", "max-carts", "repeats", "repeat-mask", "min-len", "threads", "kmer-size"], axis = 1, inplace = True)
dream_acc

,time,mem,error-code,command,bins,max-er,cmax,error-rate,bin-entropy-cutoff,repeat-period,repeat-length,matches,seconds,stellar-matches
0,0 days 00:00:31.180000,11811916,0,valik-search,1024,0.0067,254,0.0067,0.75,1000.0,NaN,22927,31.18,0
1,0 days 00:00:45.390000,13877048,0,valik-search,1024,0.0130,254,0.0130,0.95,4.0,1000.0,38553,45.39,0
2,0 days 00:01:04.260000,14216480,0,valik-search,1024,0.0200,254,0.0200,0.95,4.0,1000.0,64408,64.26,0
3,0 days 00:01:37.550000,13591332,0,valik-search,1024,0.0267,254,0.0267,0.95,1.0,1000.0,83772,97.55,103208
4,0 days 00:01:49.340000,13461092,0,valik-search,1024,0.0330,254,0.0330,0.95,1.0,1000.0,100694,109.34,119991
5,0 days 00:03:13.080000,15874196,0,valik-search,1024,0.0400,254,0.0400,0.95,1.0,1000.0,138123,193.08,160137
6,0 days 01:21:27.780000,318939592,0,valik-search,1024,0.0467,254,0.0467,0.75,1000.0,NaN,144958,4887.78,0
7,0 days 00:28:21.250000,185189236,0,valik-search,4096,0.0530,150,0.0530,0.10,1.0,1000.0,48505,1701.25,0
9,0 days 00:28:20.290000,185167760,0,valik-search,4096,0.0530,150,0.0530,0.10,1.0,1000.0,48505,1700.29,0


In [103]:
low_ent = get_highest_accuracy(dream_mouse[dream_mouse["bin-entropy-cutoff"] < 0.95])
low_ent.drop(["cart-max-cap", "max-carts", "repeats", "repeat-mask", "min-len", "threads", "kmer-size"], axis = 1, inplace = True)
low_ent

,time,mem,error-code,command,bins,max-er,cmax,error-rate,bin-entropy-cutoff,repeat-period,repeat-length,matches,seconds,stellar-matches
0,0 days 00:00:31.180000,11811916,0,valik-search,1024,0.0067,254,0.0067,0.75,1000.0,NaN,22927,31.18,0
1,0 days 00:00:35.650000,11869628,0,valik-search,1024,0.0130,254,0.0130,0.75,1000.0,NaN,32001,35.65,0
2,0 days 00:00:52.140000,12687384,0,valik-search,1024,0.0200,254,0.0200,0.75,1000.0,NaN,53067,52.14,0
3,0 days 00:01:25.360000,13771276,0,valik-search,1024,0.0267,254,0.0267,0.75,1000.0,NaN,74376,85.36,103208
4,0 days 00:01:36.760000,13682624,0,valik-search,1024,0.0330,254,0.0330,0.75,1000.0,NaN,88682,96.76,119991
5,0 days 00:02:48.900000,15561524,0,valik-search,1024,0.0400,254,0.0400,0.75,1000.0,NaN,122960,168.90,160137
6,0 days 01:21:27.780000,318939592,0,valik-search,1024,0.0467,254,0.0467,0.75,1000.0,NaN,144958,4887.78,0
7,0 days 00:28:21.250000,185189236,0,valik-search,4096,0.0530,150,0.0530,0.10,1.0,1000.0,48505,1701.25,0
9,0 days 00:28:20.290000,185167760,0,valik-search,4096,0.0530,150,0.0530,0.10,1.0,1000.0,48505,1700.29,0


In [52]:
get_fewest_matches(stellar_mouse)

,time,mem,error-code,command,min-len,error-rate,repeat-period,repeat-length,seconds,matches
0,0 days 00:15:07.040000,11697420,0,stellar-search,150,0.0267,4,25,907.04,65347
1,0 days 00:17:04.210000,11743712,0,stellar-search,150,0.0330,4,25,1024.21,91899
2,0 days 00:56:32.610000,11823132,0,stellar-search,150,0.0400,4,25,3392.61,145202


# Human vs mouse reference genomes

## Blast

In [141]:
blast_human = pd.read_csv("blast_human.time", sep = "\t")
blast_human["percid_cutoff"] = False
blast_perc = pd.read_csv("blast_human_percid.time", sep = "\t")
blast_perc["percid_cutoff"] = True

blast_human = pd.concat([blast_human, blast_perc])
blast_human['k'] = blast_human['k'].fillna(0).astype(int)
blast_human['matches'] = blast_human['matches'].fillna(0).astype(int)
blast_human = pd.merge(blast_human, error_df, how="left", on = ["k"])
blast_human = blast_human[blast_human["command"] == "blast-search"]
make_timedelta(blast_human)

blast_human.sort_values('error-rate')
blast_human.head()

,time,mem,error-code,command,threads,evalue,k,matches,percid_cutoff,error-rate,seconds
0,0 days 00:58:07.360000,3040232,0,blast-search,8.0,0.100,37,721345,False,0.0200,3487.36
1,0 days 08:03:37.650000,60693708,0,blast-search,8.0,0.100,24,192206747,False,0.0330,29017.65
2,1 days 14:41:17.100000,267744524,9,blast-search,8.0,0.100,18,0,False,0.0467,139277.10
3,0 days 00:33:17.610000,2711952,0,blast-search,8.0,0.100,51,58310,False,0.0067,1997.61
4,0 days 00:25:20.680000,2685816,0,blast-search,8.0,0.001,51,58310,False,0.0067,1520.68


In [142]:
len(blast_mouse[(blast_mouse["error-code"] != 0) & (blast_mouse["matches"] > 0)]) == 0

True

In [150]:
len(blast_mouse[(blast_mouse["error-code"] == 0) & (blast_mouse["matches"] == 0)]) == 0

True

In [153]:
blast_human[blast_human["threads"] == 8]

,time,mem,error-code,command,threads,evalue,k,matches,percid_cutoff,error-rate,seconds
0,0 days 00:58:07.360000,3040232,0,blast-search,8.0,0.1000,37,721345,False,0.0200,3487.36
1,0 days 08:03:37.650000,60693708,0,blast-search,8.0,0.1000,24,192206747,False,0.0330,29017.65
2,1 days 14:41:17.100000,267744524,9,blast-search,8.0,0.1000,18,0,False,0.0467,139277.10
3,0 days 00:33:17.610000,2711952,0,blast-search,8.0,0.1000,51,58310,False,0.0067,1997.61
4,0 days 00:25:20.680000,2685816,0,blast-search,8.0,0.0010,51,58310,False,0.0067,1520.68
5,0 days 00:37:27.510000,3193692,0,blast-search,8.0,0.0010,37,721345,False,0.0200,2247.51
6,0 days 05:48:42.820000,56265164,0,blast-search,8.0,0.0010,24,162206885,False,0.0330,20922.82
7,1 days 04:42:11.120000,509591344,9,blast-search,8.0,0.0010,16,0,False,0.0530,103331.12
9,0 days 00:24:54.090000,3281132,0,blast-search,8.0,0.0001,51,58125,False,0.0067,1494.09
10,0 days 03:21:04.350000,50610736,0,blast-search,8.0,0.0001,24,149692745,False,0.0330,12064.35


## DREAM-Stellar

In [56]:
dream_entropy_human = pd.read_csv("dream_human.time", sep = "\t")
dream_entropy_human["repeat-mask"] = "entropy"
dream_pattern_count = pd.read_csv("dream_e0.013_human.time", sep = "\t")
dream_pattern_count["repeat-mask"] = "pattern-count"
dream_human = pd.concat([dream_entropy_human, dream_pattern_count])
dream_human['kmer-size'] = dream_human['kmer-size'].fillna(0).astype(int)
dream_human['repeats'] = dream_human['repeats'].fillna(0).astype(int)
dream_human['matches'] = dream_human['matches'].fillna(0).astype(int)
dream_human.drop(["fpr", "minimiser", "repeat-flag"], axis = 1, inplace = True)
make_timedelta(dream_human)
dream_human.head()

,time,mem,error-code,command,bins,max-er,min-len,threads,cmin,cmax,error-rate,bin-entropy-cutoff,cart-max-cap,max-carts,repeats,matches,repeat-mask,kmer-size,seconds
0,0 days 09:09:29.610000,313927656,0,valik-search,2048,0.0467,150,32,0,254,0.0467,0.1,6000,2048,289358,151159,entropy,0,32969.61
1,0 days 01:06:22.690000,263526012,11,valik-search,1024,0.0467,150,32,0,254,0.0467,0.1,6000,2048,0,0,entropy,0,3982.69
2,0 days 01:08:44.670000,152865612,0,valik-search,1024,0.0267,150,32,0,254,0.0267,0.1,6000,2048,10853,69885,entropy,0,4124.67
3,0 days 03:24:06.320000,243688660,0,valik-search,2048,0.0400,150,32,0,254,0.0400,0.1,6000,2048,22744,138944,entropy,0,12246.32
4,0 days 00:13:06.450000,59084412,0,valik-search,2048,0.0200,150,32,0,254,0.0200,0.1,6000,2048,1688,46715,entropy,0,786.45


## Results

In [144]:
get_highest_accuracy(dream_human)

,time,mem,error-code,command,bins,max-er,min-len,threads,cmin,cmax,error-rate,bin-entropy-cutoff,cart-max-cap,max-carts,repeats,matches,repeat-mask,kmer-size,seconds
0,0 days 00:02:35.830000,37201924,0,valik-search,1024,0.0067,150,32,0,254,0.0067,0.1,6000,2048,1130,24342,entropy,0,155.83
1,0 days 00:20:58.900000,92523120,0,valik-search,8192,0.0130,150,32,0,254,0.0130,NaN,6000,8192,838,75929,pattern-count,19,1258.90
2,0 days 00:13:40.700000,66898128,0,valik-search,1024,0.0200,150,32,0,254,0.0200,0.1,6000,2048,2533,47198,entropy,0,820.70
3,0 days 01:08:44.670000,152865612,0,valik-search,1024,0.0267,150,32,0,254,0.0267,0.1,6000,2048,10853,69885,entropy,0,4124.67
4,0 days 02:11:17.020000,171402584,0,valik-search,2048,0.0467,150,32,0,254,0.0330,0.1,6000,2048,8164,90942,entropy,0,7877.02
5,0 days 03:45:45.140000,195607504,0,valik-search,1024,0.0400,150,32,0,254,0.0400,0.1,6000,2048,28807,146513,entropy,0,13545.14
6,0 days 09:09:29.610000,313927656,0,valik-search,2048,0.0467,150,32,0,254,0.0467,0.1,6000,2048,289358,151159,entropy,0,32969.61


In [145]:
dream_human[dream_human["seconds"] == np.min(dream_human[dream_human["error-rate"] == 0.0067]["seconds"])]

,time,mem,error-code,command,bins,max-er,min-len,threads,cmin,cmax,error-rate,bin-entropy-cutoff,cart-max-cap,max-carts,repeats,matches,repeat-mask,kmer-size,seconds
5,0 days 00:02:35.830000,37201924,0,valik-search,1024,0.0067,150,32,0,254,0.0067,0.1,6000,2048,1130,24342,entropy,0,155.83


In [146]:
dream_human[dream_human["error-rate"] == 0.0067]

,time,mem,error-code,command,bins,max-er,min-len,threads,cmin,cmax,error-rate,bin-entropy-cutoff,cart-max-cap,max-carts,repeats,matches,repeat-mask,kmer-size,seconds
5,0 days 00:02:35.830000,37201924,0,valik-search,1024,0.0067,150,32,0,254,0.0067,0.1,6000,2048,1130,24342,entropy,0,155.83
11,0 days 00:03:53.770000,34178292,0,valik-search,2048,0.0467,150,32,0,254,0.0067,0.1,6000,2048,694,23487,entropy,0,233.77


In [149]:
get_highest_accuracy(blast_human[blast_human["percid_cutoff"]])

,time,mem,error-code,command,threads,evalue,k,matches,percid_cutoff,error-rate,seconds
0,0 days 00:26:37.770000,3279712,0,blast-search,8.0,0.1000,51,1964,True,0.0067,1597.77
2,0 days 00:29:13.960000,3261376,0,blast-search,8.0,0.0001,49,3675,True,0.0130,1753.96
4,0 days 00:27:57.920000,3256892,0,blast-search,8.0,0.0001,49,3675,True,0.0130,1677.92
6,0 days 00:56:58.800000,4437324,0,blast-search,8.0,0.1000,29,688268,True,0.0267,3418.80
5,0 days 00:35:42.320000,3333976,0,blast-search,8.0,0.1000,37,54111,True,0.0200,2142.32
7,0 days 02:44:05.530000,14095552,0,blast-search,8.0,0.1000,24,1614788,True,0.0330,9845.53
8,0 days 08:24:12.940000,46427152,0,blast-search,8.0,0.1000,21,2451016,True,0.0400,30252.94


In [148]:
get_highest_accuracy(blast_human[blast_human["percid_cutoff"] == False])

,time,mem,error-code,command,threads,evalue,k,matches,percid_cutoff,error-rate,seconds
0,0 days 00:33:17.610000,2711952,0,blast-search,8.0,0.1000,51,58310,False,0.0067,1997.61
2,0 days 00:27:27.850000,3273196,0,blast-search,8.0,0.0001,49,67377,False,0.0130,1647.85
3,0 days 00:58:07.360000,3040232,0,blast-search,8.0,0.1000,37,721345,False,0.0200,3487.36
5,0 days 01:01:02.020000,7674432,0,blast-search,8.0,0.0001,29,12759097,False,0.0267,3662.02
6,0 days 08:03:37.650000,60693708,0,blast-search,8.0,0.1000,24,192206747,False,0.0330,29017.65
7,0 days 10:29:44.680000,167690752,0,blast-search,8.0,0.0001,21,483771293,False,0.0400,37784.68
8,1 days 14:41:17.100000,267744524,9,blast-search,8.0,0.1000,18,0,False,0.0467,139277.10
10,1 days 14:41:17.100000,267744524,9,blast-search,8.0,0.1000,18,0,False,0.0467,139277.10
12,1 days 04:42:11.120000,509591344,9,blast-search,8.0,0.0010,16,0,False,0.0530,103331.12


In [136]:
get_lowest_memory(dream_human)

,time,mem,error-code,command,bins,max-er,min-len,threads,cmin,cmax,error-rate,bin-entropy-cutoff,cart-max-cap,max-carts,repeats,matches,repeat-mask,kmer-size,seconds
0,0 days 00:03:53.770000,34178292,0,valik-search,2048,0.0467,150,32,0,254,0.0067,0.1,6000,2048,694,23487,entropy,0,233.77
1,0 days 00:04:02.320000,34603492,0,valik-search,2048,0.0130,150,32,0,254,0.0130,0.1,6000,2048,694,30500,entropy,0,242.32
2,0 days 00:13:06.450000,59084412,0,valik-search,2048,0.0200,150,32,0,254,0.0200,0.1,6000,2048,1688,46715,entropy,0,786.45
3,0 days 01:08:44.670000,152865612,0,valik-search,1024,0.0267,150,32,0,254,0.0267,0.1,6000,2048,10853,69885,entropy,0,4124.67
4,0 days 01:36:47.430000,152614208,11,valik-search,1024,0.0467,150,32,0,254,0.0330,0.1,6000,2048,0,0,entropy,0,5807.43
5,0 days 03:45:45.140000,195607504,0,valik-search,1024,0.0400,150,32,0,254,0.0400,0.1,6000,2048,28807,146513,entropy,0,13545.14
6,0 days 01:06:22.690000,263526012,11,valik-search,1024,0.0467,150,32,0,254,0.0467,0.1,6000,2048,0,0,entropy,0,3982.69


In [137]:
get_highest_accuracy(dream_human[dream_human["repeat-mask"] == "entropy"])

,time,mem,error-code,command,bins,max-er,min-len,threads,cmin,cmax,error-rate,bin-entropy-cutoff,cart-max-cap,max-carts,repeats,matches,repeat-mask,kmer-size,seconds
0,0 days 00:02:35.830000,37201924,0,valik-search,1024,0.0067,150,32,0,254,0.0067,0.1,6000,2048,1130,24342,entropy,0,155.83
1,0 days 00:04:02.320000,34603492,0,valik-search,2048,0.0130,150,32,0,254,0.0130,0.1,6000,2048,694,30500,entropy,0,242.32
2,0 days 00:13:40.700000,66898128,0,valik-search,1024,0.0200,150,32,0,254,0.0200,0.1,6000,2048,2533,47198,entropy,0,820.70
3,0 days 01:08:44.670000,152865612,0,valik-search,1024,0.0267,150,32,0,254,0.0267,0.1,6000,2048,10853,69885,entropy,0,4124.67
4,0 days 02:11:17.020000,171402584,0,valik-search,2048,0.0467,150,32,0,254,0.0330,0.1,6000,2048,8164,90942,entropy,0,7877.02
5,0 days 03:45:45.140000,195607504,0,valik-search,1024,0.0400,150,32,0,254,0.0400,0.1,6000,2048,28807,146513,entropy,0,13545.14
6,0 days 09:09:29.610000,313927656,0,valik-search,2048,0.0467,150,32,0,254,0.0467,0.1,6000,2048,289358,151159,entropy,0,32969.61


In [132]:
get_fastest(dream_human)

,time,mem,error-code,command,bins,max-er,min-len,threads,cmin,cmax,error-rate,bin-entropy-cutoff,cart-max-cap,max-carts,repeats,matches,repeat-mask,kmer-size,seconds
0,0 days 00:03:53.770000,34178292,0,valik-search,2048,0.0467,150,32,0,254,0.0067,0.1,6000,2048,694,23487,entropy,0,233.77
1,0 days 00:20:58.900000,92523120,0,valik-search,8192,0.0130,150,32,0,254,0.0130,NaN,6000,8192,838,75929,pattern-count,19,1258.90
2,0 days 00:13:40.700000,66898128,0,valik-search,1024,0.0200,150,32,0,254,0.0200,0.1,6000,2048,2533,47198,entropy,0,820.70
3,0 days 01:08:44.670000,152865612,0,valik-search,1024,0.0267,150,32,0,254,0.0267,0.1,6000,2048,10853,69885,entropy,0,4124.67
4,0 days 02:11:17.020000,171402584,0,valik-search,2048,0.0467,150,32,0,254,0.0330,0.1,6000,2048,8164,90942,entropy,0,7877.02
5,0 days 03:45:45.140000,195607504,0,valik-search,1024,0.0400,150,32,0,254,0.0400,0.1,6000,2048,28807,146513,entropy,0,13545.14
6,0 days 09:09:29.610000,313927656,0,valik-search,2048,0.0467,150,32,0,254,0.0467,0.1,6000,2048,289358,151159,entropy,0,32969.61
